In [1]:
import requests

In [2]:
url = "http://localhost:5555/v1/generate_sql"

headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(url, headers=headers)

print(response.text)

{"detail":"Not Found"}
